#  赛题十五 —— 论文复现自验报告

赛题内容：利用 MindQuantum 实现量子卷积神经网络求解量子多体问题

论文题目：An Application of Quantum Machine Learning on Quantum Correlated Systems: Quantum Convolutional Neural Network as a Classifier for Many-Body Wavefunctions from the Quantum Variational Eigensolver
 
复现要求：基于 MindQuantum 实现图 4 中的量子卷积神经网络，并在 N=4、8、12 的情况下实现对顺磁性和铁磁性的分类，精度要求达到 90% 以上。

## 赛题介绍

基于量子力学的量子卷积神经网络可以用来代替经典神经网络来完成分类问题。复现论文中，提出了采用量子卷积神经网络来预测某一维横场伊辛模型的基态处于顺磁相还是铁磁相。
该方案可以用于确定量子临界点 —— 这对于研究量子多体系统的相变问题具有重要意义。

一维横场伊辛模型的哈密顿量为：
$$H=-J\Sigma_{i=1}^{N}\sigma_i^z\sigma_{i+1}^z-\Gamma\Sigma_{i=1}^N\sigma_i^z.$$
其中，$\sigma_i^\alpha$ 为作用在第 $i$ 个比特上的泡利算符。$J$ 为最近邻自旋的耦合强度，此处设 $J=1$ 并将其作为系统的能量单位。系统满足周期性边界条件。当 $\Gamma>J$ 时，系统的基态处于顺磁相，而当 $\Gamma <J$ 时，系统的基态处于铁磁相。本项目就将训练量子卷积神经网络，预测不同 $\Gamma$ 值下的系统基态，是处于顺磁相还是处于铁磁相。

## 数据集

文章多用数据集为 `Tensorflow_quantum` dataset 下的 `tfi_chain` 数据集。该数据集共包括 81 个数据点，对应于所研究问题 —— 一维横场伊辛模型中的参数 $\Gamma\in[0.2,1.8]$，以及对应的构造基态的编码线路参数（4 个参数）、相信息（基态处于何种相，1 个参数，顺磁相表示为 1， 而铁磁相表示为 0）等。为训练和验证模型效果，我们将数据集随机化并拆成分别包含 60 和 21 个数据点的训练集 `train_x.npy/train_y.npy` 和验证集 `eval_x.npy/eval_y.npy`。

## 编码线路

待分类的系统基态由编码线路制备得到。下面我们以 4 比特情况为例，展示基态编码线路：


In [2]:
from mindquantum import *

qubit_num = 4
encoder = Circuit()
for i in range(qubit_num):
    encoder += H.on(i)
encoder += BarrierGate()

for i in range(qubit_num-1):
    encoder += ZZ(f'{0}').on([i, i+1])
encoder += ZZ(f'{0}').on([qubit_num-1, 0])
encoder += BarrierGate()

for i in range(qubit_num):
    encoder += RX(f'{1}').on(i)
encoder += BarrierGate()

for i in range(qubit_num-1):
    encoder += ZZ(f'{2}').on([i, i+1])
encoder += ZZ(f'{2}').on([qubit_num-1, 0])
encoder += BarrierGate()

for i in range(qubit_num):
    encoder += RX(f'{3}').on(i)
encoder += BarrierGate()

encoder

q0: ──H──‖──ZZ(0)──────────────────────ZZ(0)──‖──RX(1)──‖──ZZ(2)──────────────────────ZZ(2)──‖──RX(3)──‖
         ‖    │                          │    ‖         ‖    │                          │    ‖         ‖
q1: ──H──‖──ZZ(0)────ZZ(0)───────────────┼────‖──RX(1)──‖──ZZ(2)────ZZ(2)───────────────┼────‖──RX(3)──‖
         ‖             │                 │    ‖         ‖             │                 │    ‖         ‖
q2: ──H──‖───────────ZZ(0)────ZZ(0)──────┼────‖──RX(1)──‖───────────ZZ(2)────ZZ(2)──────┼────‖──RX(3)──‖
         ‖                      │        │    ‖         ‖                      │        │    ‖         ‖
q3: ──H──‖────────────────────ZZ(0)────ZZ(0)──‖──RX(1)──‖────────────────────ZZ(2)────ZZ(2)──‖──RX(3)──‖

该线路共包含 4 个参数（对于 8 和 12 比特情况，则包含 8 和 12 个参数）。该线路参数保存在数据集 `tarin_x` 和 `eval_x` 中。

对拟设线路，我们采用量子卷积神经网络，该线路通过施加卷积操作，将卷积核所作用的比特纠缠起来，提取信息。并通过池化操作，减少自由度，以引入非线性。最终通过比较所剩两个比特 $\sigma^z$ 算符对最终量子态的期望值大小，来作为分类依据。比如，若 $\sigma_3^z$ 期望值大于 $\sigma_5^z$ 的期望值，则规定该预测分类为 1，否则为 0。

对于一个 4 比特任务，其量子线路结构如下图所示。其中，`Conv` 表示卷积核， 而 `Pool` 表示池化操作。

![structure](./src/structure.jpg)

其他尺寸线路也为类似结构。

采用的卷积核定义为：

In [2]:
from mindquantum import *

bit_up = 0
bit_down = 1

_circ = Circuit()
_circ += RX('00').on(bit_up)
_circ += RY('01').on(bit_up)
_circ += RZ('02').on(bit_up)
_circ += RX('03').on(bit_down)
_circ += RY('04').on(bit_down)
_circ += RZ('05').on(bit_down)
_circ += ZZ('06').on([bit_up, bit_down])
_circ += YY('07').on([bit_up, bit_down])
_circ += XX('08').on([bit_up, bit_down])
_circ += RX('09').on(bit_up)
_circ += RY('10').on(bit_up)
_circ += RZ('11').on(bit_up)
_circ += RX('12').on(bit_down)
_circ += RY('13').on(bit_down)
_circ += RZ('14').on(bit_down)
_circ

q0: ──RX(00)────RY(01)────RZ(02)────ZZ(06)────YY(07)────XX(08)────RX(09)────RY(10)────RZ(11)──
                                      │         │         │
q1: ──RX(03)────RY(04)────RZ(05)────ZZ(06)────YY(07)────XX(08)────RX(12)────RY(13)────RZ(14)──

我们采用的池化操作定义为：

In [3]:
from mindquantum import *

bit_up = 0
bit_down = 1

_circ = Circuit()
_circ += RX('00').on(bit_up)
_circ += RY('01').on(bit_up)
_circ += RZ('02').on(bit_up)
_circ += RX('03').on(bit_down)
_circ += RY('04').on(bit_down)
_circ += RZ('05').on(bit_down)
_circ += X.on(bit_down, bit_up)
_circ += RX('06').on(bit_down)
_circ += RY('07').on(bit_down)
_circ += RZ('08').on(bit_down)

_circ

q0: ──RX(00)────RY(01)────RZ(02)────●────────────────────────────────
                                    │
q1: ──RX(03)────RY(04)────RZ(05)────X────RX(06)────RY(07)────RZ(08)──

优化器方面我们采用了 Adam 优化器，学习率设定为 0.01。

## 自验结果


通过执行 `4_qubit.ipynb`、`8_qubit.ipynb` 和 `12_qubit.ipynb` 中的主程序，可分别得到 4、8 和 12 比特情况下的训练结果。

对于 4 比特情况，经过大约 5.8 个回合（116 次训练）模型收敛，对验证集的分类准确度达到了 1. (参见 `4_qubit.ipynb` 中的运行结果，或 `src` 文件夹下 `result_4_qubit.png`)

![4 qubit](./src/result_4_qubit.png)


对于 8 比特情况，经过大约 5.2 个回合（105 次训练）模型收敛，对验证集的分类准确度达到了 1. （参见 `8_qubit.ipynb` 中的运行结果，或 `src` 文件夹下 `result_8_qubit.png`）

![8 qubit](./src/result_8_qubit.png)

对于 12 比特情况，经过大约 9.8 个回合（196 次训练）模型收敛，对验证集的分类准确度达到了 1. （参见 `12_qubit.ipynb` 中的运行结果，或 `src` 文件夹下的 `result_12_qubit.png`）

![12 qubit](./src/result_12_qubit.png)

## 训练

模型训练：

执行 `4_quit.ipynb` `8_qubit.ipynb` 和 `12_qubit.ipynb` 文件中的主程序，可自动对 4、8 和 12 比特情况的模型进行训练，并得到训练结果。



